In [1]:
!pip install music21


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import xml.etree.ElementTree as ET
import warnings
import music21


from music21 import *
from music21.musicxml.xmlObjects import MusicXMLImportException
from music21.musicxml.xmlToM21 import _setAttributeFromTagText

def strippedText(mxObj: ET.Element | None) -> str:
    '''
    Returns the `mxObj.text.strip()` from an Element (or None)
    taking into account that `.text` might be None, or the
    Element might be undefined.

    Replacement for the older textStripValid()

    >>> from xml.etree.ElementTree import Element
    >>> e = Element('an-element')
    >>> musicxml.xmlToM21.strippedText(e)
    ''
    >>> e.text = '    '
    >>> musicxml.xmlToM21.strippedText(e)
    ''
    >>> e.text = '  hello  '
    >>> musicxml.xmlToM21.strippedText(e)
    'hello'

    >>> musicxml.xmlToM21.strippedText(None)
    ''
    >>> musicxml.xmlToM21.strippedText(440.0)
    ''

    New in v9.
    '''
    if mxObj is None:
        return ''
    try:
        txt = mxObj.text
        if txt is None:
            return ''
        return txt.strip()
    except AttributeError:
        return ''

class MusicXMLWarning(UserWarning):
    pass


class OttomanMusicPartParser(musicxml.xmlToM21.PartParser):
    def xmlMeasureToMeasure(self, mxMeasure: ET.Element) -> stream.Measure:        
        measureParser = OttomanMusicMeasureParser(mxMeasure, parent=self)
        # noinspection PyBroadException
        try:
            measureParser.parse()
        except MusicXMLImportException as e:
            e.measureNumber = str(measureParser.measureNumber)
            e.partName = self.stream.partName
            raise e
        except Exception as e:  # pylint: disable=broad-exception-caught
            warnings.warn(
                f'The following exception took place in m. {measureParser.measureNumber} in '
                + f'part {self.stream.partName}.',
                MusicXMLWarning
            )
            raise e

        self.lastMeasureParser = measureParser

        if measureParser.staves > self.maxStaves:
            self.maxStaves = measureParser.staves

        if measureParser.transposition is not None:
            self.updateTransposition(measureParser.transposition)

        self.firstMeasureParsed = True
        self.staffReferenceList.append(measureParser.staffReference)

        m = measureParser.stream
        self.setLastMeasureInfo(m)
        # TODO: move this into the measure parsing,
        #     because it should happen on a voice level.
        if measureParser.fullMeasureRest is True:
            # recurse is necessary because it could be in voices...
            r1 = m[note.Rest].first()

            if t.TYPE_CHECKING:
                # fullMeasureRest is True, means Rest will be found
                assert r1 is not None

            if self.lastTimeSignature is not None:
                lastTSQl = self.lastTimeSignature.barDuration.quarterLength
            else:
                lastTSQl = 4.0  # sensible default.

            if (r1.fullMeasure is True  # set by xml measure='yes'
                                    or (r1.duration.quarterLength != lastTSQl
                                        and r1.duration.type in ('whole', 'breve')
                                        and r1.duration.dots == 0
                                        and not r1.duration.tuplets)):
                r1.duration.quarterLength = lastTSQl
                r1.fullMeasure = True

        # NB: not coreInsert, because barDurationProportion()
        # is called in adjustTimeAttributesFromMeasure()
        self.stream.insert(self.lastMeasureOffset, m)
        self.adjustTimeAttributesFromMeasure(m)
        # TODO: musicxml4: listening

        return m


class OttomanMusicXMLImporter(musicxml.xmlToM21.MusicXMLImporter):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def xmlPartToPart(self, mxPart, mxScorePart):
        '''
        Given a <part> object and the <score-part> object, parse a complete part.
        '''
        parser = OttomanMusicPartParser(mxPart, mxScorePart=mxScorePart, parent=self)
        parser.parse()
        if parser.appendToScoreAfterParse is True:
            return parser.stream
        else:
            return None

class OttomanMusicMeasureParser(musicxml.xmlToM21.MeasureParser):
    """
    Subclass MusicXMLImporter to allow importing non-standard accidentals
    """
    def xmlToPitch(self, mxNote, inputM21=None):
        '''
        Given a MusicXML Note object, set this Pitch object to its values.

        >>> from xml.etree.ElementTree import fromstring as EL
        >>> MP = musicxml.xmlToM21.MeasureParser()

        >>> b = EL('<note><pitch><step>E</step><alter>-1</alter>'
        ...        + '<octave>3</octave></pitch></note>')
        >>> a = MP.xmlToPitch(b)
        >>> print(a)
        E-3

        Conflicting alter and accidental -- alter is still stored, but name is :

        >>> b = EL('<note><pitch><step>E</step><alter>-1</alter><octave>3</octave></pitch>'
        ...              + '<accidental>sharp</accidental></note>')
        >>> a = MP.xmlToPitch(b)
        >>> print(a)
        E#3
        >>> a.fullName
        'E-sharp in octave 3'

        >>> a.accidental.alter
        -1.0

        >>> a.accidental.name
        'sharp'

        >>> a.accidental.modifier
        '#'
        '''
        seta = _setAttributeFromTagText
        if inputM21 is None:
            p = pitch.Pitch()
        else:
            p = inputM21

        if mxNote.tag == 'pitch':
            mxPitch = mxNote
        else:
            mxPitch = mxNote.find('pitch')
            if mxPitch is None:  # whoops!!!!
                return p

        seta(p, mxPitch, 'step')
        seta(p, mxPitch, 'octave', transform=int)
        mxAlter = mxPitch.find('alter')
        accAlter = None
        if alterText := strippedText(mxAlter):
            accAlter = float(alterText)

        mxAccidental = mxNote.find('accidental')
        mxAccidentalName = None
        if accidentalText := strippedText(mxAccidental):
            # MuseScore 0.9 made empty accidental tags for notes that did not
            # need an accidental display.
            mxAccidentalName = accidentalText

        if mxAccidentalName is not None:
            try:
                accObj = self.xmlToAccidental(mxAccidental)
                p.accidental = accObj
                p.accidental.displayStatus = True

                if accAlter is not None and accAlter != accObj.alter:
                    accObj.setAttributeIndependently('alter', float(accAlter))

            except pitch.AccidentalException:
                # MuseScore 0.9.6 generates Accidentals with empty objects
                pass
        elif accAlter is not None:
            try:
                p.accidental = pitch.Accidental(accAlter)
            except pitch.AccidentalException:
                raise MusicXMLImportException(
                    f'incorrect accidental {accAlter} for pitch {p}')
            # TODO: check supports for accidentals!
            p.accidental.displayStatus = False
        
        # !!!!!!!!!!!!!!!!!!!!
        # THIS IS THE OVERRIDE
        # !!!!!!!!!!!!!!!!!!!!
        if p.accidental is not None and p.accidental.name == 'natural':
            p.accidental = None

        return p
    
    def nonTraditionalKeySignature(self, mxKey):
        '''
        Returns a KeySignature object that represents a nonTraditional Key Signature
        '''
        # noinspection PyShadowingNames
        allChildren = list(mxKey)

        lastTag = None
        allSteps = []
        allAlters = []
        allAccidentals = []

        for c in allChildren:
            tag = c.tag
            if lastTag == 'key-alter' and tag == 'key-step':
                allAccidentals.append(None)

            if tag == 'key-step':
                allSteps.append(c.text)
            elif tag == 'key-alter':
                allAlters.append(float(c.text))
            elif tag == 'key-accidental':
                allAccidentals.append(c.text)
            lastTag = tag

        if len(allAccidentals) < len(allAlters):
            allAccidentals.append(None)
        if len(allSteps) != len(allAlters):
            raise MusicXMLImportException(
                'For non traditional signatures each step must have an alter')

        ks = key.KeySignature(sharps=None)

        alteredPitches = []
        for i in range(len(allSteps)):
            thisStep = allSteps[i]
            thisAlter = allAlters[i]
            thisAccidental = allAccidentals[i]
            p = pitch.Pitch(thisStep)
            if thisAccidental is not None:
                if thisAccidental in self.mxAccidentalNameToM21:
                    accidentalName = self.mxAccidentalNameToM21[thisAccidental]
                else:
                    accidentalName = thisAccidental
                p.accidental = OttomanMusicAccidental(accidentalName)
                p.accidental.alter = thisAlter
            else:
                p.accidental = OttomanMusicAccidental(thisAlter)
                
            alteredPitches.append(p)

        ks.alteredPitches = alteredPitches
        return ks


class OttomanMusicAccidental(music21.pitch.Accidental):
    '''
    Subclass Accidental to allow importing non-standard accidentals
    '''
    def set(self, name, *args, allowNonStandardValue=True):
        # noinspection PyShadowingNames
        super().set(name, *args, allowNonStandardValue=allowNonStandardValue)


In [3]:
"""
Parse the Teslim and extract information relevant to inferring the makam.
- input: path to the musicxml for a sarki
- return: a dictionary of the following format:
    {
    "start_note": first note of the Teslim (music21 Note object),
    "end_note": last note of the Teslim (music21 Note),
    "range": (lowest note in the Teslim, highest note of the Teslim) (pair of music21 Notes),
    "accidentals": list of notes with accidentals in the order that they appear in the Teslim (list of music21 Notes)
    }
"""
folder_path = "../With Key Signatures/"

class DoesNotHaveTeslim(Exception):
    def __init__(self):
        self.message = "There does not exist a Teslim in this piece. "
        super().__init__(self.message)

def find_teslim(filename):
    MI = OttomanMusicXMLImporter()
    score = MI.scoreFromFile(folder_path+filename)
    start_measure_num = None
    end_measure_num = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if element.content.lower() == 'teslim':
            start_measure_num =  element.getContextByClass('Measure').number
        if element.content.lower() == 'hane 2':
            end_measure_num = element.getContextByClass('Measure').number - 1
            break

    teslim = stream.Score()

    if start_measure_num is None:
        raise DoesNotHaveTeslim
    
    # Iterate through the measures in the original score and add them to the new score
    for measure in score.measures(start_measure_num, end_measure_num):
        teslim.insert(measure.offset, measure)

    return teslim

In [4]:
def extract_teslim_info(teslim):
    first_measure = teslim.parts[0].getElementsByClass('Measure')[0]
    start_note = first_measure.getElementsByClass('Note')[0]

    last_measure = teslim.parts[0].getElementsByClass('Measure')[-1]
    end_note = last_measure.getElementsByClass('Note')[-1]
    
    all_notes = teslim.parts[0].recurse().notes
    
    lowest_note = min(all_notes, key=lambda x: x.pitch)
    highest_note = max(all_notes, key=lambda x: x.pitch)
    
    accidentals = [note for note in all_notes if note.pitch.accidental is not None]
    
    result = {
        "start_note": start_note,
        "end_note": end_note,
        "range": (lowest_note, highest_note),
        "accidentals": accidentals,
        "all_pitches": set(note.pitch for note in all_notes)
    }
    return result

In [5]:
def find_maqams(filename):
    maqam_info = {}
    MI = OttomanMusicXMLImporter()
    score = MI.scoreFromFile(filename)
    start_measure_num = None
    end_measure_num = None
    maqam_name = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if start_measure_num is None:
            start_measure_num = element.getContextByClass('Measure').number
            maqam_name = element.content # the name of the next maqam
            
        elif end_measure_num is None:
            end_measure_num = element.getContextByClass('Measure').number - 1
            maqam = stream.Score()
            # Iterate through the measures in the original score and add them to the new score
            for measure in score.measures(start_measure_num, end_measure_num):
                maqam.insert(measure.offset, measure)

            maqam_list = maqam_info.get(maqam_name, [])
            maqam_list.append(extract_teslim_info(maqam))
            maqam_info[maqam_name] = maqam_list

            start_measure_num = end_measure_num + 1
            end_measure_num = None
            maqam_name = element.content
        else:
            print('something went wrong')

    return maqam_info

In [6]:
maqam_info = find_maqams('../Makam List for One Key Signature.musicxml')
{k: [d | {'accidentals': [a.nameWithOctave for a in d['accidentals']]} for d in v]for k, v in maqam_info.items()}

{'Rast': [{'start_note': <music21.note.Note D>,
   'end_note': <music21.note.Note B>,
   'range': (<music21.note.Note D>, <music21.note.Note G>),
   'accidentals': ['F#4', 'F#5', 'C#5', 'A#4'],
   'all_pitches': {<music21.pitch.Pitch D4>,
    <music21.pitch.Pitch E4>,
    <music21.pitch.Pitch F#4>,
    <music21.pitch.Pitch G4>,
    <music21.pitch.Pitch A4>,
    <music21.pitch.Pitch A#4>,
    <music21.pitch.Pitch B4>,
    <music21.pitch.Pitch C5>,
    <music21.pitch.Pitch C#5>,
    <music21.pitch.Pitch D5>,
    <music21.pitch.Pitch E5>,
    <music21.pitch.Pitch F5>,
    <music21.pitch.Pitch F#5>,
    <music21.pitch.Pitch G5>}}],
 'Pencgah': [{'start_note': <music21.note.Note G>,
   'end_note': <music21.note.Note G>,
   'range': (<music21.note.Note F#>, <music21.note.Note G>),
   'accidentals': ['C#5', 'F#5', 'A#4', 'F#4'],
   'all_pitches': {<music21.pitch.Pitch F#4>,
    <music21.pitch.Pitch G4>,
    <music21.pitch.Pitch A4>,
    <music21.pitch.Pitch A#4>,
    <music21.pitch.Pitch B4>,

In [9]:
folder_path = "../Corrected Transcriptions With Key Signatures/"

for musicfile in os.listdir(folder_path):
    try:
        teslim = find_teslim(folder_path+musicfile)
        teslim_info = extract_teslim_info(teslim)
        max_common_percent = 0
        likely_maqam = None
        for maqam_name, infos in maqam_info.items():

            for info in infos:
                common = teslim_info['all_pitches'] & info['all_pitches']
                teslim_only = teslim_info['all_pitches'] - common
                maqam_only = info['all_pitches'] - common
                common_percent = len(common) / (len(common) + len(teslim_only) + len(maqam_only))
#                 print(musicfile)
#                 print(maqam_name)
#                 print([(pitch, pitch.accidental) for pitch in teslim_only], [(pitch, pitch.accidental) for pitch in maqam_only])
#                 print(f'common: {len(common)} teslim only: {len(teslim_only)} maqam only: {len(maqam_only)}')
                if common_percent > max_common_percent:
                    max_common_percent = common_percent
                    likely_maqam = maqam_name
        print(musicfile)
        print(likely_maqam, max_common_percent)
        print()
    except Exception as e:
        print(musicfile)
        print(e)

CT 315 Hicâz ‘Aşîrân Sâz Semâ’îsi. İsma’îl Ağa’nın.musicxml
Rast 0.7692307692307693

CT 32 Sultân Selîm Hân-ı Sâlis Hazretleri'nin Pesendîde Sâz Semâ'îsi.musicxml
Rast 0.8888888888888888

CT 316-320 Acem Aşîrân Peşrevi. Tatar’ın [Muhammes].musicxml
Sazkar 0.6666666666666666

CT 23-24 Yusuf Paşa’nın Neveser Sâz Semâ’îsi.musicxml
Pesendide 0.5833333333333334

CT 128 Gerdaniye Peşrevi.musicxml
Rehavi 0.9

CT 4 Selîm Dede’nin Rehâvî Sâz Semâ’isi [Aksak Semâ’i].musicxml
Rast 1.0

CT 243-244 Muhayyer‐Kürdî Peşrevi. Neyî Râşid Efendi [Devr‐i Kebir].musicxml
Rehavi 0.6875

CT 60-62 'Osmân Bey'in Sabâ Peşrevi [Devr-i Kebir] .musicxml
Rast 0.6666666666666666

CT 38 Kemençeci Nikolaki'nin Mâhur Sâz Semâ'îsi.musicxml
Rast 0.8461538461538461

CT 76-80 İsak’ın İsfahân Peşrevi [Darb‐ı Fetih].musicxml
There does not exist a Teslim in this piece. 
CT 64-65 ‘Osmân Bey’in ‘Uşşâk Peşrevi [Muhammes].musicxml
Sazkar 0.9090909090909091

CT 322-323 Şevk ü Tarab Peşrevi. ‘Abdü’l‐kâdir Bey [Devr‐i Kebir].musicx